In [1]:
import pandas as pd
df_man = pd.read_csv('all_man_with_score.csv')

In [2]:
df_man = df_man.drop('Unnamed: 0',axis=1)

In [ ]:
#df_man

## Get reviews for all products
* Dictionary
* Product ID as keys, a list of users who provided reviews for the product

In [3]:
def get_all_user(texts):#return all user id from each product (product0, product1, .......)
    user_dict = dict()
    for i in range (len(texts.index)):
        review = eval(texts['all_reviews'][i]) 
        small_list = list()
        for x in review:
            small_list.append(x)
            user_dict[i] = small_list   
    return user_dict

In [4]:
def get_product_review(texts):#return a dictionary with each product and its all reviews
    all_customer = get_all_user(texts)
    all_review = dict()
    for i in range(len(texts.index)):#for each product
        product_review = list()
        review = eval(texts['all_reviews'][i])
        customer = all_customer[i]
        for j in range(len(customer)):#for each customer in each product
            customer_id = customer[j]
            per_review = review[customer_id][2]
            if per_review is not None:
                product_review.append(per_review)
        all_review[i] = product_review
    return all_review   

* Transform dictionary to a string
* String contains all reviews from Men category

In [5]:
def get_all_review_as_nested_list(texts):#return all allure-product's reviews
    all_customer = get_all_user(texts)
    all_review = list()
    for i in range(len(texts.index)):#for each product
        product_review = list()
        review = eval(texts['all_reviews'][i])
        customer = all_customer[i]
        for j in range(len(customer)):
            customer_id = customer[j]
            per_review = review[customer_id][2]
            if per_review is not None:
                product_review.append(per_review)
                all_review.append(per_review)
    #         all_review.append(product_review)
    return all_review 
all_review_list = get_all_review_as_nested_list(df_man)
all_review_string = ''.join(all_review_list)

In [6]:
def deEmojify(inputString):#delete all non-adcii 
    return inputString.encode('ascii', 'ignore').decode('ascii')

In [7]:
story_list = deEmojify(all_review_string)

In [8]:
#imports for LDA
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
import pprint
import nltk
from nltk.corpus import PlaintextCorpusReader
from nltk import sent_tokenize,word_tokenize 
import gensim.summarization

In [9]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from collections import OrderedDict

# LDA
## Total reviews split into 2 topics
* prepare the text
* Clean it (remove numbers, end of line characters, common words)
* Sentence tokenize it
* Convert each sentence into a list of words
* Delete some meaningless words in this context

In [10]:
def clean_data(story_list):
    sents = sent_tokenize(story_list)
    for j in range(len(sents)):
        sent = sents[j]
        sent = sent.strip().replace('\n','')
        sents[j] = sent
    story_list = '. '.join(sents)
    return story_list

In [11]:
cleaned_story_list = clean_data(story_list)
sephora_list = ['skin','use','product','face','cleanser','feel','lip','dry','lips','smell','smells','hair','scent']
texts = [[word for word in cleaned_story_list.lower().split()
          if word not in STOPWORDS and word.isalnum() and not word.lower() in sephora_list]
        ]

In [12]:
dictionary = corpora.Dictionary(texts) # (word_id,word) pairs
corpus = [dictionary.doc2bow(text) for text in texts] # (word_id,freq) pairs by sentence

In [13]:
# Set parameters
num_topics = 2 # The number of topics that should be generated
passes = 10
lda = LdaModel(corpus,
              id2word=dictionary,
              num_topics=num_topics,
              passes=passes)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(lda.print_topics(num_words=8)) # top 8 words for each

[   (   0,
        '0.018*"like" + 0.016*"love" + 0.009*"great" + 0.008*"little" + '
        '0.007*"bought" + 0.006*"good" + 0.006*"got" + 0.006*"definitely"'),
    (   1,
        '0.021*"love" + 0.018*"like" + 0.007*"great" + 0.007*"little" + '
        '0.006*"good" + 0.006*"makeup" + 0.006*"got" + 0.005*"tried"')]


In [14]:
from operator import itemgetter
lda.get_document_topics(corpus[0],minimum_probability=0.05,per_word_topics=False)
sorted(lda.get_document_topics(corpus[0],minimum_probability=0,per_word_topics=False),key=itemgetter(1),reverse=True)

[(0, 0.6299879), (1, 0.3700121)]

## Get LDA topic for every product in man

In [15]:
all_review = get_product_review(df_man)
def get_topic_df_for_products(all_review):
    import pandas as pd
    topic_for_every_product = list()
    for i in range(len(all_review)):#for each product   len(all_review)
        i_product = all_review[i]
        if i_product is not None:
            i_product_review = ''.join(i_product)
            text = i_product_review
            striptext = text.replace('\n\n', ' ')
            striptext = striptext.replace('\n', ' ')
            new_text = [nltk.word_tokenize(striptext)]
            textdictionary = corpora.Dictionary(new_text) #(word_id,frequency) pairs
            corpus_new = [dictionary.doc2bow(text) for text in new_text] #(word_id,freq) pairs by sentence
            from operator import itemgetter
            lda_topic_sorted = sorted(lda.get_document_topics(corpus_new[0],minimum_probability=0,per_word_topics=False),
                                      key=itemgetter(1),reverse=True
                               )
            topic_for_this_product = lda_topic_sorted[0][0]
            topic_for_every_product.append(topic_for_this_product)
            
    topic_for_every_product = pd.DataFrame(topic_for_every_product)
    topic_for_every_product.columns = ['LDA topic']    
    return topic_for_every_product


* find topic for each product

In [16]:
topic_df = get_topic_df_for_products(all_review)
df_man['LDA_topic'] = topic_df['LDA topic']
df_man.to_csv('man_with_lda_topic.csv', index=False)

In [17]:
df_man[df_man['LDA_topic']==1]

,product_id,brand_name,product_name,category,subcategory,loves,price,total_review,avg_rating,recommended_count,onestar_count,twostar_count,threestar_count,fourstar_count,fivestar_count,all_reviews,vader_100_score,LDA_topic
0,P431738,CLINIQUE,Clinique For Men™ Anti-Aging Starter Kit,Men,Value & Gift Sets,2232,$14.50,1,5.0000,1,0,0,0,0,1,"{'118463861': [5, 'Very nice product', 'Really...",0.926800,1
11,P429628,Kilian,"Straight To Heaven, White Cristal Travel Spray...",Fragrance,Value & Gift Sets,693,$195.00,4,5.0000,4,0,0,0,0,4,"{'111419722': [5, None, 'So excited to see Kil...",0.883200,1
13,P429713,DOLCE&GABBANA,Dolce Gift Set,Fragrance,Value & Gift Sets,1783,$124.00($178.00,4,5.0000,4,0,0,0,0,4,"{'117182195': [5, None, 'Light, floral, refres...",0.729050,1
19,P429076,HERMÈS,Eau de citron noir Eau de Cologne,Fragrance,Women,834,$95.00,8,4.6250,7,0,1,0,0,7,"{'125398173': [5, 'Very Sexy', 'I wear Jour d’...",0.718575,1
21,P375131,HERMÈS,Eau de pamplemousse rose,Fragrance,Women,1612,$130.00,14,4.3571,4,1,0,0,5,8,"{'42101290': [4, 'Love it!', ""I'am a perfume a...",0.622343,1
25,P165958,Ralph Lauren,Polo Double Black,Fragrance,Men,1465,$68.00,39,4.7436,1,0,0,0,10,29,"{'25511157': [5, 'YUM!', ""Bought this for my b...",0.701323,1
27,P163604,Acqua Di Parma,Colonia,Fragrance,Men,2480,$63.00,74,4.2973,6,5,4,6,8,51,"{'30348669': [5, 'Fresh, sophisticated, elegan...",0.726123,1
32,P245710,TOM FORD,Grey Vetiver Eau de Parfum,Fragrance,Men,2523,$128.00,49,4.5714,7,1,4,1,3,40,"{'20161038': [5, 'Classic', ""Don't buy this wi...",0.705604,1
33,P307802,Acqua Di Parma,Bergamotto di Calabria,Fragrance,Women,1827,$113.00,16,4.5000,6,0,0,3,2,11,"{'24382185': [5, 'Beautiful and Unique', ""This...",0.726194,1
37,P416049,Eight & Bob,Cap D’Antibes,Fragrance,Women,166,$175.00,1,2.0000,0,0,1,0,0,0,"{'121683124': [2, 'Misleading Description', ""T...",-0.541100,1


In [18]:
df_man[df_man['LDA_topic']==0]

,product_id,brand_name,product_name,category,subcategory,loves,price,total_review,avg_rating,recommended_count,onestar_count,twostar_count,threestar_count,fourstar_count,fivestar_count,all_reviews,vader_100_score,LDA_topic
1,P416960,CHANEL,BLEU DE CHANEL Eau de Parfum Twist and Spray R...,Fragrance,Value & Gift Sets,1242,$100.00,6,4.1667,5,1,0,0,1,4,"{'126440744': [1, 'not what i expected disappo...",0.505517,0
2,P414107,Jack Black,Carry On and Travel Fresh,Men,Value & Gift Sets,1524,$15.00,2,5.0000,0,0,0,0,0,2,"{'79950757': [5, 'mmmmm', 'Great price- smells...",0.464800,0
3,P443384,Kiehl's Since 1851,Men's Skincare Starter Kit,Skincare,Value & Gift Sets,1835,$39.00,3,5.0000,3,0,0,0,0,3,"{'126474089': [5, 'Kiehl’s Men’s skincare star...",0.798333,0
4,P225364,The Art of Shaving,The 4 Elements of the Perfect Shave™ Carry-On ...,Men,Value & Gift Sets,2093,$60.00,44,4.7273,3,0,1,1,7,35,"{'20558612': [4, 'Smells amazing!', 'This stuf...",0.806255,0
5,P429620,Kilian,"Love, Don't be Shy Travel Spray Set",Fragrance,Value & Gift Sets,3448,$195.00,7,4.5714,6,0,1,0,0,6,"{'119048445': [5, 'Broken Cap', 'My last revie...",0.347443,0
6,P429618,Kilian,Intoxicated Travel Spray Set,Fragrance,Value & Gift Sets,504,$195.00,1,4.0000,1,0,0,0,1,0,"{'129354261': [4, 'love the smell', 'I absolut...",0.048000,0
7,P429626,Kilian,"Black Phantom - ""Memento Mori"" Travel Spray Set",Fragrance,Value & Gift Sets,899,$195.00,1,5.0000,1,0,0,0,0,1,"{'113292153': [5, None, 'Sweet with light smok...",0.740200,0
8,P431739,CLINIQUE,Clinique For Men™ Water-Gel Starter Kit,Men,Value & Gift Sets,1727,$14.50,3,4.6667,3,0,0,0,1,2,"{'118463725': [5, 'Amazing products', ""Bought ...",0.718700,0
9,P425225,DOLCE&GABBANA,Light Blue Eau Intense Pour Homme Gift Set,Fragrance,Value & Gift Sets,1183,$91.00($124.00,3,5.0000,3,0,0,0,0,3,"{'114350799': [5, 'He loves it', 'Got this for...",0.589567,0
10,P431737,CLINIQUE,Clinique For Men™ Oil Control Starter Kit,Men,Value & Gift Sets,1259,$14.50,2,4.5000,2,0,0,0,1,1,"{'122880067': [5, 'Great kit!', 'I bought this...",0.546000,0
